# 🎯 SD 1.5 Prompting Techniques for Jewelry Generation
## Based on Machine Learning Mastery Article

This notebook implements various prompting techniques from the Machine Learning Mastery article to improve jewelry image generation using **Stable Diffusion 1.5**. Each technique is tested on all 8 Arcade assignment prompts with 6 different seeds.

## 🎨 Techniques Tested:
1. **Baseline Prompts** - Original prompts without enhancement
2. **Medium Enhancement** - Adding photography medium specifications  
3. **Artistic Style** - Adding artistic style keywords
4. **Famous Artists** - Including renowned artist names
5. **Website References** - Adding platform names (ArtStation, etc.)
6. **Resolution Enhancement** - High-quality descriptors (4K, sharp focus, etc.)
7. **Lighting Enhancement** - Professional lighting techniques
8. **Color Enhancement** - Specific color guidance
9. **Negative Prompts** - What NOT to include
10. **Keyword Emphasis** - Using weighting factors

## 📊 Output Format:
- **8 rows × 7 columns grid** for each technique
- **Row 1-8**: Each jewelry prompt from Arcade assignment
- **Column 1**: Prompt text
- **Columns 2-7**: 6 generated images with different seeds


In [ ]:
# Setup and Imports
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from datetime import datetime
import random

# Set up device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Load SD 1.5 pipeline
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False
)
pipe = pipe.to(device)

# Use DPM++ 2M Karras sampler as recommended in article
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Enable memory efficient attention
pipe.enable_attention_slicing()

# Try to enable xformers if available, fallback gracefully if not
try:
    if device == "cuda":
        pipe.enable_xformers_memory_efficient_attention()
        print("✅ xformers memory optimization enabled")
except Exception as e:
    print(f"⚠️ xformers not available, using standard attention: {str(e)}")
    # Use alternative memory optimizations
    if hasattr(pipe, 'enable_model_cpu_offload'):
        pipe.enable_model_cpu_offload()
        print("✅ CPU offloading enabled as alternative")

print("✅ SD 1.5 pipeline loaded successfully!")

# Create output directory
output_dir = "sd15_prompting_results"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# Main Ultimate Testing Loop
print("🚀 Starting ULTIMATE comprehensive experiment...")
print(f"⏱️ Testing {len(MODEL_CONFIGS)} models × {len(SAMPLER_CONFIGS)} samplers × {len(STEP_COUNTS)} steps × {len(CFG_SCALES)} CFG scales × {len(STRATEGY_CONFIGS)} strategies × {len(test_prompts)} prompts")

# Store all results with enhanced structure
all_results = {}
current_pipe = None
current_config = None
generation_counter = 0

start_time = datetime.now()
print(f"🕐 Experiment started at: {start_time}")

# Test each combination systematically
for model_choice in MODEL_CONFIGS.keys():
    print(f"\n🤖 Testing model: {model_choice}")
    
    for sampler_choice in SAMPLER_CONFIGS.keys():
        print(f"\n  🎛️ Testing sampler: {sampler_choice}")
        
        # Load pipeline with current sampler (reuse if same config)
        config_key = f"{model_choice}_{sampler_choice}"
        if current_config != config_key:
            if current_pipe is not None:
                del current_pipe
                torch.cuda.empty_cache()
            current_pipe = load_model_with_sampler(model_choice, sampler_choice)
            current_config = config_key
        
        for steps in STEP_COUNTS:
            print(f"\n    📊 Testing {steps} steps...")
            
            for cfg_scale in CFG_SCALES:
                print(f"\n      ⚙️ Testing CFG scale {cfg_scale}...")
                
                for strategy_name in STRATEGY_CONFIGS.keys():
                    strategy_config = STRATEGY_CONFIGS[strategy_name]
                    print(f"\n        🎯 Testing strategy: {strategy_name}")
                    
                    for prompt_idx, original_prompt in enumerate(test_prompts, 1):
                        generation_counter += 1
                        
                        # Apply strategy to prompt
                        modified_prompt = strategy_config["modifier"](original_prompt)
                        
                        print(f"          📝 Prompt {prompt_idx}/8 ({generation_counter:,}/{total_combinations:,}): {original_prompt[:40]}...")
                        
                        # Generate image
                        image, gen_time, error = generate_with_config(
                            current_pipe, modified_prompt, model_choice, steps, cfg_scale,
                            seed=100 + prompt_idx  # Consistent seed per prompt
                        )
                        
                        if image is not None:
                            # Enhanced evaluation with both CLIP and aesthetic scores
                            clip_results = analyze_image_with_clip(image)
                            aesthetic_score = get_aesthetic_score(image)
                            
                            # Create comprehensive filename
                            filename = f"combined_experiment_results/{model_choice}_{sampler_choice}_{steps}s_{cfg_scale}cfg_{strategy_name}_p{prompt_idx:02d}.png"
                            image.save(filename)
                            
                            # Store comprehensive result
                            result_key = f"{model_choice}_{sampler_choice}_{steps}_{cfg_scale}_{strategy_name}_{prompt_idx}"
                            all_results[result_key] = {
                                'model': model_choice,
                                'sampler': sampler_choice,
                                'steps': steps,
                                'cfg_scale': cfg_scale,
                                'strategy': strategy_name,
                                'prompt_id': prompt_idx,
                                'original_prompt': original_prompt,
                                'modified_prompt': modified_prompt,
                                'image': image,
                                'filepath': filename,
                                'generation_time': gen_time,
                                'clip_top_label': clip_results[0][0],
                                'clip_top_confidence': clip_results[0][1],
                                'clip_results': clip_results,
                                'laion_aesthetic_score': aesthetic_score,
                                'error': None,
                                'timestamp': datetime.now()
                            }
                            
                            print(f"            ✅ Generated in {gen_time:.1f}s | CLIP: {clip_results[0][0]} ({clip_results[0][1]:.3f}) | Aesthetic: {aesthetic_score:.2f}")
                            
                        else:
                            print(f"            ❌ Failed: {error}")
                            result_key = f"{model_choice}_{sampler_choice}_{steps}_{cfg_scale}_{strategy_name}_{prompt_idx}"
                            all_results[result_key] = {
                                'model': model_choice,
                                'sampler': sampler_choice,
                                'steps': steps,
                                'cfg_scale': cfg_scale,
                                'strategy': strategy_name,
                                'prompt_id': prompt_idx,
                                'original_prompt': original_prompt,
                                'modified_prompt': modified_prompt,
                                'image': None,
                                'filepath': None,
                                'generation_time': gen_time,
                                'error': error,
                                'timestamp': datetime.now()
                            }
                        
                        # Progress tracking
                        if generation_counter % 100 == 0:
                            elapsed = datetime.now() - start_time
                            avg_time_per_gen = elapsed.total_seconds() / generation_counter
                            remaining_time = (total_combinations - generation_counter) * avg_time_per_gen / 3600
                            print(f"\n📈 Progress: {generation_counter:,}/{total_combinations:,} ({generation_counter/total_combinations*100:.1f}%)")
                            print(f"⏱️ Elapsed: {elapsed} | Est. remaining: {remaining_time:.1f} hours")

# Cleanup
if current_pipe is not None:
    del current_pipe
    torch.cuda.empty_cache()

end_time = datetime.now()
total_time = end_time - start_time

print(f"\n🎉 ULTIMATE EXPERIMENT COMPLETED!")
print(f"⏱️ Total time: {total_time}")
successful_results = sum(1 for r in all_results.values() if r.get('image') is not None)
total_results = len(all_results)
print(f"📊 Results: {successful_results:,}/{total_results:,} successful generations ({successful_results/total_results*100:.1f}%)")


In [ ]:
# Main Ultimate Testing Loop
print("🚀 Starting ULTIMATE comprehensive experiment...")
print(f"⏱️ Testing {len(MODEL_CONFIGS)} models × {len(SAMPLER_CONFIGS)} samplers × {len(STEP_COUNTS)} steps × {len(CFG_SCALES)} CFG scales × {len(STRATEGY_CONFIGS)} strategies × {len(test_prompts)} prompts")

# Store all results with enhanced structure
all_results = {}
current_pipe = None
current_config = None
generation_counter = 0

start_time = datetime.now()
print(f"🕐 Experiment started at: {start_time}")

# Test each combination systematically
for model_choice in MODEL_CONFIGS.keys():
    print(f"\n🤖 Testing model: {model_choice}")
    
    for sampler_choice in SAMPLER_CONFIGS.keys():
        print(f"\n  🎛️ Testing sampler: {sampler_choice}")
        
        # Load pipeline with current sampler (reuse if same config)
        config_key = f"{model_choice}_{sampler_choice}"
        if current_config != config_key:
            if current_pipe is not None:
                del current_pipe
                torch.cuda.empty_cache()
            current_pipe = load_model_with_sampler(model_choice, sampler_choice)
            current_config = config_key
        
        for steps in STEP_COUNTS:
            print(f"\n    📊 Testing {steps} steps...")
            
            for cfg_scale in CFG_SCALES:
                print(f"\n      ⚙️ Testing CFG scale {cfg_scale}...")
                
                for strategy_name in STRATEGY_CONFIGS.keys():
                    strategy_config = STRATEGY_CONFIGS[strategy_name]
                    print(f"\n        🎯 Testing strategy: {strategy_name}")
                    
                    for prompt_idx, original_prompt in enumerate(test_prompts, 1):
                        generation_counter += 1
                        
                        # Apply strategy to prompt
                        modified_prompt = strategy_config["modifier"](original_prompt)
                        
                        print(f"          📝 Prompt {prompt_idx}/8 ({generation_counter:,}/{total_combinations:,}): {original_prompt[:40]}...")
                        
                        # Generate image
                        image, gen_time, error = generate_with_config(
                            current_pipe, modified_prompt, model_choice, steps, cfg_scale,
                            seed=100 + prompt_idx  # Consistent seed per prompt
                        )
                        
                        if image is not None:
                            # Enhanced evaluation with both CLIP and aesthetic scores
                            clip_results = analyze_image_with_clip(image)
                            aesthetic_score = get_aesthetic_score(image)
                            
                            # Create comprehensive filename
                            filename = f"combined_experiment_results/{model_choice}_{sampler_choice}_{steps}s_{cfg_scale}cfg_{strategy_name}_p{prompt_idx:02d}.png"
                            image.save(filename)
                            
                            # Store comprehensive result
                            result_key = f"{model_choice}_{sampler_choice}_{steps}_{cfg_scale}_{strategy_name}_{prompt_idx}"
                            all_results[result_key] = {
                                'model': model_choice,
                                'sampler': sampler_choice,
                                'steps': steps,
                                'cfg_scale': cfg_scale,
                                'strategy': strategy_name,
                                'prompt_id': prompt_idx,
                                'original_prompt': original_prompt,
                                'modified_prompt': modified_prompt,
                                'image': image,
                                'filepath': filename,
                                'generation_time': gen_time,
                                'clip_top_label': clip_results[0][0],
                                'clip_top_confidence': clip_results[0][1],
                                'clip_results': clip_results,
                                'laion_aesthetic_score': aesthetic_score,
                                'error': None,
                                'timestamp': datetime.now()
                            }
                            
                            print(f"            ✅ Generated in {gen_time:.1f}s | CLIP: {clip_results[0][0]} ({clip_results[0][1]:.3f}) | Aesthetic: {aesthetic_score:.2f}")
                            
                        else:
                            print(f"            ❌ Failed: {error}")
                            result_key = f"{model_choice}_{sampler_choice}_{steps}_{cfg_scale}_{strategy_name}_{prompt_idx}"
                            all_results[result_key] = {
                                'model': model_choice,
                                'sampler': sampler_choice,
                                'steps': steps,
                                'cfg_scale': cfg_scale,
                                'strategy': strategy_name,
                                'prompt_id': prompt_idx,
                                'original_prompt': original_prompt,
                                'modified_prompt': modified_prompt,
                                'image': None,
                                'filepath': None,
                                'generation_time': gen_time,
                                'error': error,
                                'timestamp': datetime.now()
                            }
                        
                        # Progress tracking
                        if generation_counter % 100 == 0:
                            elapsed = datetime.now() - start_time
                            avg_time_per_gen = elapsed.total_seconds() / generation_counter
                            remaining_time = (total_combinations - generation_counter) * avg_time_per_gen / 3600
                            print(f"\n📈 Progress: {generation_counter:,}/{total_combinations:,} ({generation_counter/total_combinations*100:.1f}%)")
                            print(f"⏱️ Elapsed: {elapsed} | Est. remaining: {remaining_time:.1f} hours")

# Cleanup
if current_pipe is not None:
    del current_pipe
    torch.cuda.empty_cache()

end_time = datetime.now()
total_time = end_time - start_time

print(f"\n🎉 ULTIMATE EXPERIMENT COMPLETED!")
print(f"⏱️ Total time: {total_time}")
successful_results = sum(1 for r in all_results.values() if r.get('image') is not None)
total_results = len(all_results)
print(f"📊 Results: {successful_results:,}/{total_results:,} successful generations ({successful_results/total_results*100:.1f}%)")


In [ ]:
# Main Ultimate Testing Loop
print("🚀 Starting ULTIMATE comprehensive experiment...")
print(f"⏱️ Testing {len(MODEL_CONFIGS)} models × {len(SAMPLER_CONFIGS)} samplers × {len(STEP_COUNTS)} steps × {len(CFG_SCALES)} CFG scales × {len(STRATEGY_CONFIGS)} strategies × {len(test_prompts)} prompts")

# Store all results with enhanced structure
all_results = {}
current_pipe = None
current_config = None
generation_counter = 0

start_time = datetime.now()
print(f"🕐 Experiment started at: {start_time}")

# Test each combination systematically
for model_choice in MODEL_CONFIGS.keys():
    print(f"\n🤖 Testing model: {model_choice}")
    
    for sampler_choice in SAMPLER_CONFIGS.keys():
        print(f"\n  🎛️ Testing sampler: {sampler_choice}")
        
        # Load pipeline with current sampler (reuse if same config)
        config_key = f"{model_choice}_{sampler_choice}"
        if current_config != config_key:
            if current_pipe is not None:
                del current_pipe
                torch.cuda.empty_cache()
            current_pipe = load_model_with_sampler(model_choice, sampler_choice)
            current_config = config_key
        
        for steps in STEP_COUNTS:
            print(f"\n    📊 Testing {steps} steps...")
            
            for cfg_scale in CFG_SCALES:
                print(f"\n      ⚙️ Testing CFG scale {cfg_scale}...")
                
                for strategy_name in STRATEGY_CONFIGS.keys():
                    strategy_config = STRATEGY_CONFIGS[strategy_name]
                    print(f"\n        🎯 Testing strategy: {strategy_name}")
                    
                    for prompt_idx, original_prompt in enumerate(test_prompts, 1):
                        generation_counter += 1
                        
                        # Apply strategy to prompt
                        modified_prompt = strategy_config["modifier"](original_prompt)
                        
                        print(f"          📝 Prompt {prompt_idx}/8 ({generation_counter:,}/{total_combinations:,}): {original_prompt[:40]}...")
                        
                        # Generate image
                        image, gen_time, error = generate_with_config(
                            current_pipe, modified_prompt, model_choice, steps, cfg_scale,
                            seed=100 + prompt_idx  # Consistent seed per prompt
                        )
                        
                        if image is not None:
                            # Enhanced evaluation with both CLIP and aesthetic scores
                            clip_results = analyze_image_with_clip(image)
                            aesthetic_score = get_aesthetic_score(image)
                            
                            # Create comprehensive filename
                            filename = f"combined_experiment_results/{model_choice}_{sampler_choice}_{steps}s_{cfg_scale}cfg_{strategy_name}_p{prompt_idx:02d}.png"
                            image.save(filename)
                            
                            # Store comprehensive result
                            result_key = f"{model_choice}_{sampler_choice}_{steps}_{cfg_scale}_{strategy_name}_{prompt_idx}"
                            all_results[result_key] = {
                                'model': model_choice,
                                'sampler': sampler_choice,
                                'steps': steps,
                                'cfg_scale': cfg_scale,
                                'strategy': strategy_name,
                                'prompt_id': prompt_idx,
                                'original_prompt': original_prompt,
                                'modified_prompt': modified_prompt,
                                'image': image,
                                'filepath': filename,
                                'generation_time': gen_time,
                                'clip_top_label': clip_results[0][0],
                                'clip_top_confidence': clip_results[0][1],
                                'clip_results': clip_results,
                                'laion_aesthetic_score': aesthetic_score,
                                'error': None,
                                'timestamp': datetime.now()
                            }
                            
                            print(f"            ✅ Generated in {gen_time:.1f}s | CLIP: {clip_results[0][0]} ({clip_results[0][1]:.3f}) | Aesthetic: {aesthetic_score:.2f}")
                            
                        else:
                            print(f"            ❌ Failed: {error}")
                            result_key = f"{model_choice}_{sampler_choice}_{steps}_{cfg_scale}_{strategy_name}_{prompt_idx}"
                            all_results[result_key] = {
                                'model': model_choice,
                                'sampler': sampler_choice,
                                'steps': steps,
                                'cfg_scale': cfg_scale,
                                'strategy': strategy_name,
                                'prompt_id': prompt_idx,
                                'original_prompt': original_prompt,
                                'modified_prompt': modified_prompt,
                                'image': None,
                                'filepath': None,
                                'generation_time': gen_time,
                                'error': error,
                                'timestamp': datetime.now()
                            }
                        
                        # Progress tracking
                        if generation_counter % 100 == 0:
                            elapsed = datetime.now() - start_time
                            avg_time_per_gen = elapsed.total_seconds() / generation_counter
                            remaining_time = (total_combinations - generation_counter) * avg_time_per_gen / 3600
                            print(f"\n📈 Progress: {generation_counter:,}/{total_combinations:,} ({generation_counter/total_combinations*100:.1f}%)")
                            print(f"⏱️ Elapsed: {elapsed} | Est. remaining: {remaining_time:.1f} hours")

# Cleanup
if current_pipe is not None:
    del current_pipe
    torch.cuda.empty_cache()

end_time = datetime.now()
total_time = end_time - start_time

print(f"\n🎉 ULTIMATE EXPERIMENT COMPLETED!")
print(f"⏱️ Total time: {total_time}")
successful_results = sum(1 for r in all_results.values() if r.get('image') is not None)
total_results = len(all_results)
print(f"📊 Results: {successful_results:,}/{total_results:,} successful generations ({successful_results/total_results*100:.1f}%)")


In [ ]:
# Arcade Assignment Jewelry Prompts
arcade_prompts = [
    "channel-set diamond eternity band, 2 mm width, hammered 18k yellow gold, product-only white background",
    "solitaire diamond engagement ring, round brilliant cut 1 carat, classic 6-prong setting, platinum band, product-only white background", 
    "three-stone engagement ring, emerald-cut center diamond with trilliant side stones, rose gold setting, product-only white background",
    "vintage art deco engagement ring, cushion-cut diamond with milgrain detailing, white gold band, product-only white background",
    "halo engagement ring, oval diamond surrounded by smaller diamonds, split shank band, yellow gold, product-only white background",
    "tennis bracelet, round diamonds in 4-prong settings, 18k white gold, 7 inches long, product-only white background",
    "diamond stud earrings, round brilliant cut, 4-prong settings, 18k yellow gold, product-only white background",
    "pearl necklace, cultured freshwater pearls, 18-inch length, sterling silver clasp, product-only white background"
]

print("📝 Loaded 8 Arcade Assignment Jewelry Prompts:")
for i, prompt in enumerate(arcade_prompts, 1):
    print(f"{i}. {prompt}")

# Generation parameters (following article recommendations)
generation_params = {
    "width": 512,
    "height": 512,
    "num_inference_steps": 20,  # Article recommends 20-40 steps
    "guidance_scale": 7,  # CFG scale of 7
    "num_images_per_prompt": 1
}

# Seeds for reproducibility (6 different seeds per technique per prompt)
seeds = [42, 123, 456, 789, 999, 2024]

print(f"\n⚙️ Generation Parameters:")
for key, value in generation_params.items():
    print(f"  {key}: {value}")
print(f"  Seeds: {seeds}")


In [ ]:
# Utility Functions

def generate_images(prompt, technique_name, negative_prompt=""):
    """Generate 6 images for a prompt using different seeds"""
    images = []
    
    for i, seed in enumerate(seeds):
        print(f"    Generating image {i+1}/6 (seed: {seed})...")
        
        # Set seed
        generator = torch.Generator(device=device).manual_seed(seed)
        
        # Generate image
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            generator=generator,
            **generation_params
        )
        
        images.append(result.images[0])
    
    return images

def create_grid_display(technique_name, prompts, all_images):
    """Create and display 8x7 grid: 8 prompts × (1 text + 6 images)"""
    fig, axes = plt.subplots(8, 7, figsize=(21, 24))
    fig.suptitle(f'{technique_name}', fontsize=20, fontweight='bold', y=0.98)
    
    for row in range(8):
        prompt = prompts[row]
        images = all_images[row]
        
        # Column 0: Display prompt text
        axes[row, 0].text(0.05, 0.5, f"Prompt {row+1}:\\n{prompt[:100]}..." if len(prompt) > 100 else f"Prompt {row+1}:\\n{prompt}",
                         fontsize=8, ha='left', va='center', wrap=True, transform=axes[row, 0].transAxes)
        axes[row, 0].axis('off')
        
        # Columns 1-6: Display generated images
        for col in range(1, 7):
            if col-1 < len(images):
                axes[row, col].imshow(images[col-1])
                axes[row, col].set_title(f"Seed {seeds[col-1]}", fontsize=8)
            axes[row, col].axis('off')
    
    plt.tight_layout()
    
    # Save the grid
    save_path = os.path.join(output_dir, f"{technique_name.lower().replace(' ', '_')}_grid.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"💾 Grid saved to: {save_path}")
    
    plt.show()

def run_technique_experiment(technique_name, prompt_modifier_func, negative_prompt=""):
    """Run a complete experiment for one technique"""
    print(f"\\n🎨 {technique_name.upper()}")
    print("=" * 60)
    
    all_images = []
    
    for i, base_prompt in enumerate(arcade_prompts):
        print(f"\\n📿 Prompt {i+1}: {base_prompt[:50]}...")
        
        # Apply technique-specific prompt modification
        enhanced_prompt = prompt_modifier_func(base_prompt)
        print(f"🔧 Enhanced: {enhanced_prompt[:80]}...")
        
        # Generate images for this prompt
        images = generate_images(enhanced_prompt, technique_name, negative_prompt)
        all_images.append(images)
    
    # Create and display grid
    create_grid_display(technique_name, 
                       [prompt_modifier_func(p) for p in arcade_prompts], 
                       all_images)
    
    return all_images

print("✅ Utility functions defined!")


## 🔬 Technique 1: Baseline Prompts
**Original prompts without any enhancement - establishing the control group**


In [ ]:
# Technique 1: Baseline - No modifications
def baseline_modifier(prompt):
    return prompt

baseline_results = run_technique_experiment("Technique 1: Baseline Prompts", baseline_modifier)


## 🔬 Technique 2: Medium Enhancement
**Adding photography medium specifications to make prompts more specific**
- Ultra-realistic photography
- Professional product photography
- High-end jewelry photography


In [ ]:
# Technique 2: Medium Enhancement
def medium_modifier(prompt):
    return f"Ultra-realistic professional jewelry photography of {prompt}"

medium_results = run_technique_experiment("Technique 2: Medium Enhancement", medium_modifier)


## 🔬 Technique 3: Artistic Style Enhancement  
**Adding artistic style keywords to influence the aesthetic**
- Hyperrealistic style
- Luxury commercial photography style
- Modern minimalist aesthetic


In [ ]:
# Technique 3: Artistic Style Enhancement
def artistic_style_modifier(prompt):
    return f"Hyperrealistic luxury commercial photography style, modern minimalist aesthetic, {prompt}"

artistic_results = run_technique_experiment("Technique 3: Artistic Style Enhancement", artistic_style_modifier)


## 🔬 Technique 4: Famous Artists Enhancement
**Including renowned photographer/artist names for style influence**
- Annie Leibovitz (famous portrait photographer)
- Irving Penn (master of still life photography)  
- Richard Avedon (iconic fashion photographer)


In [ ]:
# Technique 4: Famous Artists Enhancement
def artist_modifier(prompt):
    return f"{prompt}, by Annie Leibovitz and Irving Penn, in the style of Richard Avedon"

artist_results = run_technique_experiment("Technique 4: Famous Artists Enhancement", artist_modifier)


## 🔬 Technique 5: Website References Enhancement
**Adding platform names that are known for high-quality imagery**
- ArtStation (digital art platform)
- Behance (creative portfolio platform)
- Professional photography websites


In [ ]:
# Technique 5: Website References Enhancement
def website_modifier(prompt):
    return f"{prompt}, ArtStation, Behance, professional jewelry photography portfolio"

website_results = run_technique_experiment("Technique 5: Website References Enhancement", website_modifier)


## 🔬 Technique 6: Resolution Enhancement
**Adding high-quality descriptors for detailed output**
- 4K, 8K resolution specifications
- Sharp focus, highly detailed
- Professional quality indicators


In [ ]:
# Technique 6: Resolution Enhancement
def resolution_modifier(prompt):
    return f"{prompt}, 4K, 8K, highly detailed, sharp focus, professional quality, ultra-detailed"

resolution_results = run_technique_experiment("Technique 6: Resolution Enhancement", resolution_modifier)


## 🔬 Technique 7: Lighting Enhancement
**Adding professional lighting techniques for better visual appeal**
- Rim lighting, studio lighting
- Cinematic lighting, soft lighting
- Professional jewelry photography lighting


In [ ]:
# Technique 7: Lighting Enhancement
def lighting_modifier(prompt):
    return f"{prompt}, professional studio lighting, rim lighting, soft diffused lighting, cinematic lighting"

lighting_results = run_technique_experiment("Technique 7: Lighting Enhancement", lighting_modifier)


## 🔬 Technique 8: Color Enhancement
**Adding specific color guidance for better color accuracy**
- Brilliant colors, vibrant colors
- True-to-life metal tones
- Accurate gemstone colors


In [ ]:
# Technique 8: Color Enhancement
def color_modifier(prompt):
    return f"{prompt}, brilliant colors, vibrant, true-to-life metal tones, accurate color representation"

color_results = run_technique_experiment("Technique 8: Color Enhancement", color_modifier)


## 🔬 Technique 9: Negative Prompts Enhancement
**Using negative prompts to specify what should NOT be in the image**
Based on the article's recommended negative prompt template for high-quality outputs.


In [ ]:
# Technique 9: Negative Prompts Enhancement
def negative_modifier(prompt):
    return prompt

# Comprehensive negative prompt from the article, adapted for jewelry
jewelry_negative_prompt = """(worst quality, low quality, normal quality, low-res, low details, oversaturated, undersaturated, overexposed, underexposed, grayscale, bw, bad photo, bad photography, bad art:1.4), (watermark, signature, text font, username, error, logo, words, letters, digits, autograph, trademark, name:1.2), (blur, blurry, grainy), ugly, asymmetrical, mutated malformed, mutilated, poorly lit, bad shadow, draft, cropped, out of frame, cut off, censored, jpeg artifacts, out of focus, glitch, duplicate, (airbrushed, cartoon, anime, semi-realistic, CGI, render, blender, digital art, manga, amateur:1.3), (bad hands, bad anatomy, bad jewelry, deformed jewelry, fake looking, plastic, cheap:1.3), multiple objects, cluttered, messy background"""

negative_results = run_technique_experiment("Technique 9: Negative Prompts Enhancement", negative_modifier, jewelry_negative_prompt)


## 🔬 Technique 10: Keyword Emphasis Enhancement
**Using weighting factors to emphasize important keywords**
Using the article's (keyword:factor) syntax to emphasize crucial jewelry terms.


In [ ]:
# Technique 10: Keyword Emphasis Enhancement
def emphasis_modifier(prompt):
    # Add emphasis to important jewelry terms using (keyword:factor) syntax
    emphasized_prompt = prompt.replace("diamond", "(diamond:1.3)")
    emphasized_prompt = emphasized_prompt.replace("gold", "(gold:1.2)")
    emphasized_prompt = emphasized_prompt.replace("platinum", "(platinum:1.2)")
    emphasized_prompt = emphasized_prompt.replace("engagement ring", "(engagement ring:1.2)")
    emphasized_prompt = emphasized_prompt.replace("product-only white background", "(product-only white background:1.4)")
    return emphasized_prompt

emphasis_results = run_technique_experiment("Technique 10: Keyword Emphasis Enhancement", emphasis_modifier, jewelry_negative_prompt)


## 🎯 Experiment Summary
**Comprehensive results from all 10 prompting techniques**


In [ ]:
print("🎉 EXPERIMENT COMPLETE!")
print("=" * 60)
print("✅ Generated 480 images total (8 prompts × 6 seeds × 10 techniques)")
print("✅ Created 10 comprehensive grids (8×7 layout)")
print("✅ Tested all major prompting techniques from Machine Learning Mastery article")

print("\\n📁 Generated Files:")
print(f"📂 Output directory: {output_dir}")
print("📄 Grid images saved as: technique_name_grid.png")

print("\\n🔍 Techniques Tested:")
techniques = [
    "1. Baseline Prompts",
    "2. Medium Enhancement", 
    "3. Artistic Style Enhancement",
    "4. Famous Artists Enhancement",
    "5. Website References Enhancement",
    "6. Resolution Enhancement",
    "7. Lighting Enhancement", 
    "8. Color Enhancement",
    "9. Negative Prompts Enhancement",
    "10. Keyword Emphasis Enhancement"
]

for technique in techniques:
    print(f"   ✓ {technique}")

print("\\n🎯 Key Insights:")
print("• Compare grids to see which techniques work best for jewelry")
print("• Look for improved detail, lighting, and realism")
print("• Notice differences in background quality and product focus")
print("• Evaluate which approaches produce most professional results")

print("\\n📊 Next Steps:")
print("• Analyze the generated grids visually")
print("• Identify the most effective techniques")
print("• Combine the best techniques for optimal prompts")
print("• Apply learnings to future jewelry generation tasks")
